In [ ]:
!pip install yfinance ta scikit-learn matplotlib seaborn plotly


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import ta  # Technical Analysis library

# Optional: suppress warnings to keep output clean
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")


In [ ]:
# Define the ticker and the data period
ticker = 'AAPL'
data = yf.download(ticker, period='6mo', interval='1d')

# Check if data is fetched
if data.empty:
    raise ValueError("No data fetched. Check the ticker, period, or your network connection.")

data.dropna(inplace=True)  # Remove any rows with missing values
print(f"Fetched {len(data)} rows for {ticker}")
data.head()


In [ ]:
# Calculate technical indicators using the ta library and flatten the output

# Make sure data['Close'] is a Series (it usually is, but .squeeze() ensures 1D)
close_series = data['Close'].squeeze()

# Calculate and flatten the outputs before creating a Series aligned with the index
sma_values = ta.trend.sma_indicator(close_series, window=20).values.flatten()
ema_values = ta.trend.ema_indicator(close_series, window=20).values.flatten()
rsi_values = ta.momentum.rsi(close_series, window=14).values.flatten()

# Assign back to the DataFrame, ensuring the index is preserved
data['SMA_20'] = pd.Series(sma_values, index=data.index)
data['EMA_20'] = pd.Series(ema_values, index=data.index)
data['RSI_14'] = pd.Series(rsi_values, index=data.index)

# Drop any rows with NaN values that may have been created by the indicator calculations
data.dropna(inplace=True)

print("Technical indicators added.")
data.head()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name='Candlestick'
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['SMA_20'],
    mode='lines',
    name='SMA 20'
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['EMA_20'],
    mode='lines',
    name='EMA 20'
))

fig.update_layout(
    title=f"{ticker} Price Chart with Technical Indicators",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    xaxis_rangeslider_visible=False,
    height=600
)

fig.show()


In [ ]:
# Create a new target column: next day’s close
data['Target'] = data['Close'].shift(-1)

# Drop the last row as it will have a missing Target
data = data.dropna()

# Select features and label
features = data[['Open', 'High', 'Low', 'Volume', 'SMA_20', 'EMA_20', 'RSI_14']]
label = data['Target']

print(f"Features shape: {features.shape}, Label shape: {label.shape}")
data.tail()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, shuffle=False)
print("Training set:", X_train.shape, y_train.shape)
print("Testing set:", X_test.shape, y_test.shape)


In [ ]:
# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Random Forest model trained!")


In [ ]:
# Make predictions on test data
y_pred = model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='Actual Price', linewidth=2)
plt.plot(y_test.index, y_pred, label='Predicted Price', linestyle='--', linewidth=2)
plt.title(f"{ticker} - Actual vs Predicted Closing Prices")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Use the last available data point from the features
latest_features = features.iloc[-1].values.reshape(1, -1)
next_day_prediction = model.predict(latest_features)[0]
print(f"Predicted next closing price for {ticker}: ${next_day_prediction:.2f}")
